In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
from random import sample
import argparse
import numpy as np
import os
import pickle
from tqdm import tqdm
from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis
from scipy.ndimage import gaussian_filter
from skimage import morphology
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import matplotlib

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.models import wide_resnet50_2, resnet18

import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [4]:
class ImageDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, self.image_files[idx]  # Renvoie l'image et son nom

In [ ]:
# Create a new train file with just two classes
filepath = ""
train_dataset = pd.read_csv(filepath)
train_dataset = train_dataset.replace("GOOD", "nm")
train_dataset =train_dataset.replace("Boucle plate", "nm")
train_dataset =train_dataset.replace("Lift-off blanc", "nm")
train_dataset =train_dataset.replace("Lift-off noir", "nm")
train_dataset =train_dataset.replace("Short circuit MOS", "nm")

train_dataset.to_csv("modified_train.csv")


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
train_dataset = ImageDataset(csv_file="/Users/nicolasthiou/Desktop/DataChallenge/Y_train_eVW9jym.csv", root_dir="/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_train", transform=transform, train=True)
test_dataset = ImageDataset(csv_file="/Users/nicolasthiou/Desktop/DataChallenge/Y_random_nKwalR1.csv", root_dir="/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_test", transform=transform, train=False)


Classifier 1, decides if missing or not missing

In [ ]:
#Train classifier 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

import torchvision.models as models
import torch.nn as nn

# Load Pretrained ResNet50
model = models.resnet50(pretrained=True)

num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

import torch.optim as optim

# Move model to GPU if available
device = torch.device("mps")
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")

In [ ]:
classifier_file1 = '/var/AI_Models_Training/TorchServe_essai/Benchmark_Challenge_ENS/Benoit/Version Finale/Classifier.pt'
image_folder = '/var/AI_Models_Training/TorchServe_essai/Benchmark_Challenge_ENS/Benoit/Dataset/input_test'

# Transformation des images pour l'entrée du modèle
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

# Chargement du dataset
dataset = ImageDataset(image_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

# Load classifier model
model = torch.load(classifier_file1)
model.to(device)
model.eval()  

# Passer les images dans le modèle
y_pred_classifier1 = []
filenames_list = []
output_classifier=[]


with torch.no_grad():  # Désactive le calcul des gradients
    for images, filenames in dataloader:
        images = images.to(device)
        outputs = model(images)
        # Move the outputs to CPU and convert to numpy
        output_classifier.extend(outputs.cpu().numpy())
        filenames_list.extend(filenames)  # Ajouter les noms de fichiers

# Store scores and filenames
for filename, score in zip(filenames_list, output_classifier):
    y_pred_classifier1.append((filename, score))

Classifier 2, decides the class if not missing

In [ ]:
classifier_file2 = '/var/AI_Models_Training/TorchServe_essai/Benchmark_Challenge_ENS/Benoit/Version Finale/Classifier.pt'
image_folder = '/var/AI_Models_Training/TorchServe_essai/Benchmark_Challenge_ENS/Benoit/Dataset/input_test'

# Transformation des images pour l'entrée du modèle
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

# Chargement du dataset
dataset = ImageDataset(image_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

# Load classifier model
model = torch.load(classifier_file1)
model.to(device)
model.eval()  

# Passer les images dans le modèle
y_pred_classifier2 = []
filenames_list = []
output_classifier=[]


with torch.no_grad():  # Désactive le calcul des gradients
    for images, filenames in dataloader:
        images = images.to(device)
        outputs = model(images)
        # Move the outputs to CPU and convert to numpy
        output_classifier.extend(outputs.cpu().numpy())
        filenames_list.extend(filenames)  # Ajouter les noms de fichiers

# Store scores and filenames
for filename, score in zip(filenames_list, output_classifier):
    y_pred_classifier2.append((filename, score))

Anomaly detector, again only if not missing

In [ ]:
test_path = '/var/AI_Models_Training/TorchServe_essai/Benchmark_Challenge_ENS/Benoit/Dataset/input_test'
padim_trained = 'C:/Users/yagoa/Documents/MASH/Mallat/data_challenge/models_GXqjjM/PADIM.pkl'

class CustomDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.images = []


        for img_name in os.listdir(self.data_path):
            img_path = os.path.join(self.data_path, img_name)
            self.images.append(img_path)


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")  # Charger l'image
    
        # Appliquer les transformations
        if self.transform:
            image = self.transform(image)  # Appliquer les transformations à l'image

        filename = os.path.basename(img_path)

        return filename, image                               


def denormalization(x):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    x = (((x.transpose(1, 2, 0) * std) + mean) * 255.).astype(np.uint8)
    
    return x


def embedding_concat(x, y):
    B, C1, H1, W1 = x.size()
    _, C2, H2, W2 = y.size()
    s = int(H1 / H2)
    x = F.unfold(x, kernel_size=s, dilation=1, stride=s)
    x = x.view(B, C1, -1, H2, W2)
    z = torch.zeros(B, C1 + C2, x.size(2), H2, W2)
    for i in range(x.size(2)):
        z[:, :, i, :, :] = torch.cat((x[:, :, i, :, :], y), 1)
    z = z.view(B, -1, H2 * W2)
    z = F.fold(z, kernel_size=s, output_size=(H1, W1), stride=s)

    return z


# Charger les données d'entraînement
with open(padim_trained, 'rb') as f:
    train_outputs = pickle.load(f)

model = wide_resnet50_2(pretrained=True, progress=True)
t_d = 1792
d = 550


# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

model.to(device)
model.eval()
random.seed(1024)
torch.manual_seed(1024)
if use_cuda:
    torch.cuda.manual_seed_all(1024)

idx = torch.tensor(sample(range(0, t_d), d))

# set model's intermediate outputs
outputs = []

def hook(module, input, output):
    outputs.append(output)

model.layer1[-1].register_forward_hook(hook)
model.layer2[-1].register_forward_hook(hook)
model.layer3[-1].register_forward_hook(hook)


transform = transforms.Compose([
                                transforms.Resize((128, 128)),  # Redimensionner les images
                                transforms.ToTensor(),            # Convertir en tenseur
                                ])

test_dataset = CustomDataset(test_path, transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, pin_memory=True)
test_outputs = OrderedDict([('layer1', []), ('layer2', []), ('layer3', [])])

test_imgs = []
filenames_list = []
scores_list = []

# extract test set features
for filename, x in tqdm(test_dataloader, '| feature extraction | test |'):
    filenames_list.extend(filename)  # Collect filenames for the batch
    test_imgs.extend(x.cpu().detach().numpy())
    # model prediction
    with torch.no_grad():
        _ = model(x.to(device))
    # get intermediate layer outputs
    for k, v in zip(test_outputs.keys(), outputs):
        test_outputs[k].append(v.cpu().detach())
    # initialize hook outputs
    outputs = []
for k, v in test_outputs.items():
    test_outputs[k] = torch.cat(v, 0)

# Embedding concat
embedding_vectors = test_outputs['layer1']
for layer_name in ['layer2', 'layer3']:
    embedding_vectors = embedding_concat(embedding_vectors, test_outputs[layer_name])

# randomly select d dimension
embedding_vectors = torch.index_select(embedding_vectors, 1, idx)

# calculate distance matrix
B, C, H, W = embedding_vectors.size()
embedding_vectors = embedding_vectors.view(B, C, H * W).numpy()
dist_list = []
for i in range(H * W):
    mean = train_outputs[0][:, i]
    conv_inv = np.linalg.inv(train_outputs[1][:, :, i])
    dist = [mahalanobis(sample[:, i], mean, conv_inv) for sample in embedding_vectors]
    dist_list.append(dist)

dist_list = np.array(dist_list).transpose(1, 0).reshape(B, H, W)

# upsample
dist_list = torch.tensor(dist_list)
score_map = F.interpolate(dist_list.unsqueeze(1), size=x.size(2), mode='bilinear',
                            align_corners=False).squeeze().numpy()

# apply gaussian smoothing on the score map
for i in range(score_map.shape[0]):
    score_map[i] = gaussian_filter(score_map[i], sigma=4)

# Normalization
max_score = score_map.max()
min_score = score_map.min()
scores = (score_map - min_score) / (max_score - min_score)

# calculate image-level ROC AUC score
img_scores = scores.reshape(scores.shape[0], -1).max(axis=1)

# Store scores and filenames
for filename, score in zip(filenames_list, img_scores):
    scores_list.append((filename, score))
    
y_pred_anomaly = scores_list

Decision process


In [ ]:
# Convertir y_pred_anomaly en un dictionnaire
anomaly_dict = dict(y_pred_anomaly)

# Convertir y_pred_classifier en un dictionnaire
classifier_dict1 = dict(y_pred_classifier1)
classifier_dict2 = dict(y_pred_classifier2)

# Fusionner les deux dictionnaires basés sur le filename
# Assurer que tous les filenames sont présents dans les deux modèles
merged_data = []
for filename in anomaly_dict.keys():
    if filename in classifier_dict2 and filename in classifier_dict1:
        anomaly_score = anomaly_dict[filename]
        classifier_probs2 = classifier_dict2[filename]
        classifier_probs1 = classifier_dict1[filename]
        merged_data.append((filename, anomaly_score, *classifier_probs1, *classifier_probs2))  # Fusionner les données
        

# Créer un DataFrame pandas avec les résultats
final_df = pd.DataFrame(merged_data, columns=['filename', 'p_drift', 'p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p_missing'])

merged_df = pd.merge(final_df, test_df, on='filename', how='inner')

def define_classe(row, threshold=0.5):
    p_missing = row['p_missing']
    # Accéder à la valeur de p_drift dans la ligne
    p_drift = row['p_drift']
    
    # Extraire les probabilités des classes (colonnes 2 à 7)
    p_classes = row[['p0', 'p1', 'p2', 'p3', 'p4', 'p5']].values
    
    if p_missing > 0.6:
        return 4
    if p_drift > threshold:
        return 6
    else:
        return p_classes.argmax()  # Utiliser argmax pour obtenir l'indice du max

# Appliquer la fonction à chaque ligne du DataFrame
merged_df['y_pred'] = merged_df.apply(define_classe, axis=1)